# Generating GW signal to build dedicated simulation

This notebook explains how generate GW signal (h+,hx, projected strain, TDI) from source parameters using the LDC toolbox.  

## Software requirements

- git clone https://gitlab.in2p3.fr/LISA/LDC
- pip install requirements.txt
- python setup.py build_liborbits install

Look at the LDC README for more details.

- git clone https://gitlab.in2p3.fr/j2b.bayle/LISANode.git
- pip install -e .

## Generating a Galactic binary signal

In [ ]:
import matplotlib.pyplot as plt
import scipy
import numpy as np
import xarray as xr
from astropy import units as u
import pandas as pd

import ldc.io.hdf5 as hdfio
from ldc.waveform.source import load_gb_catalog, load_mbhb_catalog
from ldc.lisa.noise import get_noise_model
from ldc.lisa.orbits import Orbits
from ldc.lisa.projection import ProjectedStrain
from ldc.common.series import TimeSeries, FrequencySeries
import ldc.waveform.fastGB as fastGB
from ldc.waveform.waveform import HpHc
from LISAhdf5 import LISAhdf5,ParsUnits

Let's use a verification binary from the catalog provided by the LDC software

In [ ]:

DATAPATH = "/home/stefan/LDC/Radler/data"
sangria_fn = DATAPATH + "/dgb-tdi.h5"
sangria_fn = DATAPATH + "/LDC1-3_VGB_v2_FD_noiseless.hdf5"
FD5 = LISAhdf5(sangria_fn)
Nsrc = FD5.getSourcesNum()
GWs = FD5.getSourcesName()
print("Found %d GW sources: " % Nsrc, GWs)
### TOD make sure GalBin is there
if GWs[0] != "GalBinaries":
    raise NotImplementedError
p = FD5.getSourceParameters(GWs[0])
td = FD5.getPreProcessTDI()
del_t = float(p.get("Cadence"))
Tobs = float(p.get("ObservationDuration"))

### Window function used in fft
def window(tm, offs=1000.0):

    xl = offs
    ind_r = np.argwhere(tm[-1]-tm <= offs)[0][0]
    xr = tm[ind_r]
    # print (xr)
    kap = 0.005
    winl = 0.5*(1.0 + np.tanh(kap*(tm-xl)))
    winr = 0.5*(1.0 - np.tanh(kap*(tm-xr)))
    # plt.plot(tm, winl)
    # plt.plot(tm, winr)
    # plt.grid(True)
    # plt.show()
    return (winl*winr)


We choose the 1st source and set observation parameters

In [ ]:

parameters = [
    "Amplitude",
    "EclipticLatitude",
    "EclipticLongitude",
    "Frequency",
    "FrequencyDerivative",
    "Inclination",
    "InitialPhase",
    "Polarization",
]
pGB = {}
ind = 8
for parameter in parameters:
    pGB[parameter] = p.get(parameter)[ind]
print('pGB', pGB)

dt = del_t
print(dt)
# dt = 5 # waveform sampling
t_max = 60*60*24*365 # time of observation = 1yr
print(t_max)
t_min = 0
t_max = Tobs
print(t_max)

config = {"initial_position": 0, "initial_rotation": 0, 
          "nominal_arm_length": 2500000000, "orbit_type": 'analytic'}
lisa_orbits = Orbits.type(config)

### Generating noise free TDI X,Y,Z using fast waveform generator

In [ ]:
GB = fastGB.FastGB(delta_t=dt, T=t_max, orbits=lisa_orbits) # in seconds
Xs, Ys, Zs = GB.get_fd_tdixyz(template=pGB, oversample=4, simulator='synthlisa')

plt.figure(figsize=(15, 5))
for j, d in enumerate([Xs, Ys, Zs]):
    plt.subplot(1,3,j+1)
    plt.plot(d.f, d)
    plt.xlabel("Time [s]")
    if j==0:
        plt.ylabel("TDI")
        plt.legend()

### Generating noise free projected strain

In the following, projector.yArm (nt, 6) contains the projected strain signal for the 6 links, it can be saved in an h5 file to be given as input to LISANode. The LISANode configuration file has to be changed for:

```
LISA_GW_TYPE = 'file'
LISA_GW_FILE_FS = 0.2 #1 over dt 
```

Then: ```lisanode run lisanode:LISAWithTDI --gw-path my_gw_for_lisanode.h5```

The actual call to lisanode is shown below. 

In [ ]:
GB = HpHc.type('AM_CVn', "GB", "TD_fdot")
GB.set_param(pGB)
projector = ProjectedStrain(lisa_orbits)
yArm = projector.arm_response(t_min, t_max, dt, GB.split())
projector.links

In [ ]:
projector.to_file("my_gw_for_lisanode.h5")

### From projected strain to TDI

In the following, we illustrate 3 ways of producing noise free TDI from the projected strain: 
- using the LDC time domain interpolated TDI
- using pyTDI
- using LISANode

which should all give the same result as the fast TDI code shown above. 

In [ ]:
trange = np.arange(t_min, t_max, dt)
tdi_X = projector.compute_tdi_x(trange)


In [ ]:
p = FD5.getSourceParameters(GWs[0])
td = FD5.getPreProcessTDI()
del_t = float(p.get("Cadence"))
Tobs = float(p.get("ObservationDuration"))

dt = del_t

# Build timeseries and frequencyseries object for X,Y,Z
tdi_ts = xr.Dataset(dict([(k, TimeSeries(td[:, n], dt=dt)) for k, n in [["X", 1], ["Y", 2], ["Z", 3]]]))
# tdi_ts = xr.Dataset(dict([(k,TimeSeries(tdi_ts[k][:,1], dt=dt)) for k in ["X", "Y", "Z"]]))
tdi_fs = xr.Dataset(dict([(k, tdi_ts[k].ts.fft(win=window)) for k in ["X", "Y", "Z"]]))

In [ ]:

tdi_X_td = TimeSeries(tdi_X, dt=dt)
tdi_X_fd = tdi_X_td.ts.fft(win=window)
plt.figure(figsize=(15,10))
sl = slice(Xs.f.min(),Xs.f.max())
plt.plot(Xs.f, Xs, label="fast waveform")
plt.plot(tdi_X_fd.f, tdi_X_fd, label="strain to TDI", alpha=0.5)
plt.plot(tdi_fs['X'].f, tdi_fs['X'], label="data", alpha=0.5)
plt.xlabel("Time [s]")
plt.ylabel("TDI X")
plt.xlim(Xs.f.min()+0.000007,Xs.f.max()-0.000007)

### Using pyTDI

In [ ]:
pGB = {'Amplitude': 6.373545164051205e-23, 'EclipticLatitude': -0.08222513098415238, 'EclipticLongitude': 2.1022552527635017, 'Frequency': 0.006220281478962882, 'FrequencyDerivative': 7.486504954481661e-16, 'Inclination': 0.6452098954792331, 'InitialPhase': 3.1415926535329115, 'Polarization': 2.9013532283888614}
GB = fastGB.FastGB(delta_t=dt, T=t_max, orbits=lisa_orbits) # in seconds
Xs, Ys, Zs = GB.get_fd_tdixyz(template=pGB, oversample=4, simulator='synthlisa')

plt.figure(figsize=(15, 5))
for j, d in enumerate([Xs, Ys, Zs]):
    plt.subplot(1,3,j+1)
    plt.plot(d.f, d)
    plt.xlabel("Time [s]")
    if j==0:
        plt.ylabel("TDI")
        plt.legend()

GB = HpHc.type('AM_CVn', "GB", "TD_fdot")
GB.set_param(pGB)
projector = ProjectedStrain(lisa_orbits)
yArm = projector.arm_response(t_min, t_max, dt, GB.split())
projector.links
trange = np.arange(t_min, t_max, dt)
tdi_X = projector.compute_tdi_x(trange)
tdi_X_td = TimeSeries(tdi_X, dt=dt)
tdi_X_fd = tdi_X_td.ts.fft(win=window)

plt.figure(figsize=(15,10))
sl = slice(Xs.f.min(),Xs.f.max())
plt.plot(Xs.f, Xs, label="fast waveform")
plt.plot(tdi_X_fd.f, tdi_X_fd, label="strain to TDI", alpha=0.5)
plt.plot(tdi_fs['X'].f, tdi_fs['X'], label="data", alpha=0.5)
plt.xlabel("Time [s]")
plt.ylabel("TDI X")
plt.xlim(Xs.f.min()+0.000007,Xs.f.max()-0.000007)

In [ ]:
from pytdi.michelson import X1_ETA as X
data = {}
traveltimes = {}
for j, n in enumerate(projector.links):
    r,s = int(n[0]), int(n[-1])
    data[f"eta_{r}{s}"] = yArm[:,j]
    traveltimes[f"d_{r}{s}"] = lisa_orbits.compute_travel_time(s,r,trange) 
fs = 1/dt
data_X = X.build(data, traveltimes, fs, order=(3, 3))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(trange, data_X, label="strain to TDI using pyTDI")
plt.plot(trange, tdi_X, label="strain to TDI using LDC", alpha=0.5)
plt.xlabel("Time [s]")
plt.ylabel("TDI X")
plt.axis([1000, None, -1.5e-23, None])
plt.legend()
plt.subplot(122)
sl = slice(100000,100500)
plt.plot(trange[sl], data_X[sl], label="strain to TDI using pyTDI")
plt.plot(trange[sl], tdi_X[sl], label="strain to TDI using LDC", alpha=0.5)
plt.xlabel("Time [s]")
plt.ylabel("TDI X")
plt.axis([trange[sl.start]-100, trange[sl.stop]+100, -1.5e-23, None])

In [ ]:
del projector, yArm, tdi_X, X, traveltimes, data, data_X, trange

## Calling LISANode

In the following, we call LISANode from here and load the resulting TDI output. 
Beware that the default configuration file is changed by this process. 

### Note about Orbits

One should use the same orbits and travel time for the projection and in the simulator. 
For now, this can be achieved by adjusting the LISANode configuration to : 

```
LISA_ORBIT_TYPE = 'ldc'
```

are running with the ```flags``` option to link LISANode to the LDC orbits and constants libraries as shown below.

Not all orbits parameters are propagated through the full LISA graph, such that it's not easy to change from the default ones. 

In a near future (LDC2b production), orbits file generated with the [lisaorbits](https://gitlab.in2p3.fr/lisa-simulation/orbits) software could be used on both sides.  


### Note about LISANode frequency sampling

For the time being, the LISANode compilation takes a lot of time and memory due to the sampling difference between input GW strain (0.2Hz) and LISA measurement (30Hz).  

This issue should be solved soon (again LDC2b production), with GW strain reading using the same buffering system as the one developed for reading orbits file. 


In [ ]:
import lisanode.lisa
import os
dest = lisanode.lisa.__file__.replace('__init__', 'config')
os.system("cp %s %s"%("config.py", dest))

ldc_path = "/home/maude/soft/LDC/ldc/lisa/orbits" # set the path to LDC
flags = f"-L{ldc_path}/lib -I{ldc_path}/lib -I{ldc_path}/../../common/constants -lorbits"

output_file = "gb-tdi.h5"
ln_cmd = f"lisanode run --enable X Y Z --flags='{flags}' lisanode:LISAWithTDI -d 10000 -o {output_file} --gw-path my_gw_for_lisanode.h5"
ln_cmd += " --lasernoise-on-off 0 --backlinknoise-on-off 0 --telescopenoise-on-off 0 --accelnoise-on-off 0 --readoutnoise-on-off 0 --obpathlengthnoise-on-off 0 --usonoise-on-off 0 --modulationnoise-on-off 0 --rangingnoise-on-off 0"
ln_cmd
os.system(ln_cmd)

In [ ]:
X_sim, attrs = hdfio.load_array("gb-tdi.h5")

sl = slice(0,int(10000/dt))
plt.figure()
plt.plot(Xs.t[sl], Xs[sl], label="strain to TDI using pyTDI")
plt.plot(X_sim["X"][:,0], X_sim["X"][:,1], label="lisanode", alpha=0.5)
plt.xlabel("Time [s]")
plt.ylabel("TDI X")
plt.axis([0, 10000, -1.5e-23, 1.5e-23])
plt.legend()


## Generating a massive black hole merger signal

In [ ]:
cat, units = load_mbhb_catalog("/home/maude/soft/LDC/data_generation/test/data/catalog_1_yrs_full_1.dat")
pd.DataFrame(cat)

In [ ]:
s_index = 3 # take the fourth, which merge before 1 yr 
pMBHB = dict(zip(cat.dtype.names, cat[s_index]))
pMBHB["ObservationDuration"] = t_max
pMBHB["Cadence"] = dt
MBHB = HpHc.type("demo", "MBHB", "IMRPhenomD")
MBHB.set_param(pMBHB)
projector = ProjectedStrain(lisa_orbits)
yArm = projector.arm_response(t_min, t_max, dt, MBHB.split())
projector.to_file("my_mbhb_for_lisanode.h5")

In [ ]:
trange = np.arange(t_min, t_max, dt)
tdi_X = projector.compute_tdi_x(trange)

plt.figure()
plt.plot(trange, tdi_X, label="strain to TDI", alpha=0.5)
plt.xlabel("Time [s]")
plt.ylabel("TDI X")
plt.legend()